# Cognito フェデレーション Identity を使用した Identity 分離を備えた Amazon Bedrock AgentCore Runtime と AgentCore Memory エージェント

## 概要

このチュートリアルでは、Cognito フェデレーション Identity を使用した Amazon Bedrock AgentCore Memory での安全な Memory 分離の実装方法を説明します。Amazon Cognito Identity Pool からの認証されたユーザー資格情報に基づいて会話履歴を自動的にパーティション化し分離する Memory 対応エージェントを構築し、マルチテナント環境でのデータプライバシーとセキュリティを確保します。

Memory の分離は、本番会話エージェントにとって重要なセキュリティ要件です。適切な分離がないと、ユーザーが他のユーザーの会話履歴にアクセスできる可能性があり、プライバシー違反やデータ漏洩につながります。

この実装は、AgentCore Memory が Amazon Cognito Identity Pool と統合して、フェデレーションユーザー資格情報に基づいて Memory 境界を自動的に適用する方法を示しています。アプリケーションコードでカスタム認可ロジックを不要にします。

### チュートリアル詳細

| 項目              | 詳細                                                             |
|-------------------|------------------------------------------------------------------|
| チュートリアルタイプ | セキュリティと Identity 管理                                      |
| エージェントタイプ   | 単一会話エージェント                                              |
| エージェントフレームワーク | Strands Agents                                              |
| LLM モデル         | Anthropic Claude 4.5 Haiku                                       |
| 主な機能           | Memory 分離、フェデレーション Identity、ユーザーコンテキスト         |
| 使用 SDK          | boto3、bedrock-agentcore、bedrock-agentcore-starter-toolkit      |

### 学習内容

このチュートリアルでは、以下を学習します：
1. AgentCore Memory でフェデレーション Identity 用の Cognito Identity Pool を設定する方法
2. 一時的な AWS 資格情報を通じてユーザー Identity を伝播する方法
3. 安全なデータパーティショニングのためのフェデレーション資格情報を持つ Memory フックを実装する方法
4. 分離された Memory スペースを持つマルチユーザーエージェントをデプロイ・テストする方法
5. フェデレーション Identity を使用して異なるユーザー間の Memory 分離を検証する方法


### アーキテクチャ

この例は、フェデレーション Identity ベースの Memory 分離を備えた AgentCore Runtime にデプロイされた会話エージェントを示しています：

<div style="text-align:left">
    <img src="architecture.png" width="90%"/>
</div>

## 0. 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10 以降
* Bedrock、ECR、IAM、Cognito に対する適切な権限を持つ AWS 認証情報
* Amazon Bedrock モデルアクセス（Claude 3.5 Haiku）
* Amazon Bedrock AgentCore SDK と依存関係

まず、必要なライブラリをインストールしましょう。

In [ ]:
!pip install -qUr requirements.txt

### 環境のセットアップ

必要なライブラリをインポートし、環境を設定しましょう。以下を使用します：
- `boto3`：AWS サービスとのやり取り用
- `bedrock_agentcore.memory`：エージェント Memory の管理用
- 認証設定用の各種ユーティリティ関数

In [ ]:
# Imports
import os
import boto3
import uuid
import logging
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from utils import setup_cognito_user_pool, create_agentcore_role

# Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("runtime-memory-agent")
REGION = os.getenv('AWS_REGION', 'us-west-2')

## 1. Memory リソースの作成

このセクションでは、エージェントが会話履歴を保存するための Memory リソースを作成します。Memory により、エージェントは過去のインタラクションを思い出し、コンテキストを維持し、時間の経過とともにより一貫性のある応答を提供できます。

この例では、追加の長期ストラテジーを持たないシンプルな短期 Memory リソースを作成します。Memory はすべての会話メッセージを保存し、AgentCore Runtime でセッションが終了した後も会話を継続する際にエージェントが以前のインタラクションを記憶するのに役立ちます。

In [ ]:
# Create unique identifier for this resource
unique_id = str(uuid.uuid4())[:8]
memory_name = f"RuntimeIdentityMemoryAgent_{unique_id}"

# Initialize Memory Manager
memory_manager = MemoryManager(region_name=REGION)

# Create memory
print("\n🧠 Creating memory...")
print("   This takes 2-3 minutes...\n")

memory = memory_manager.get_or_create_memory(
    name=memory_name,
    strategies=[],
    description="Memory isolation with IAM example.",
    event_expiry_days=30  # Optional: adjust as needed
)

MEMORY_ID = memory.get("id")
print(f"\n✅ Memory created successfully!")
print(f"   Memory ID: {MEMORY_ID}")
print(f"   Status: {memory.get('status')}")

## 2. Amazon Cognito User Pool と Identity Pool の作成

このセクションでは、Amazon Cognito User Pool、Identity Pool、およびユーザーを作成します。Cognito はエージェントにユーザー認証と Identity 管理を提供し、フェデレーション Identity 資格情報を通じて各ユーザーの会話履歴がそのユーザーのみがアクセスできることを保証します。

`setup_cognito_user_pool` 関数は以下を行います：
1. Cognito User Pool が存在しない場合は作成
2. フェデレーション Identity 用の Cognito Identity Pool を作成
3. 認証用のアプリクライアントを設定
4. 一時パスワードを持つ 2 人のテストユーザーを作成
5. テスト用のアクセストークンと ID トークンを生成

In [ ]:
print("Setting up Amazon Cognito user pool and users...")
cognito_config = setup_cognito_user_pool(REGION,MEMORY_ID)
print("Cognito setup completed ✓")

## 3. Memory 対応エージェントの作成

このセクションでは、Memory 統合用のカスタムフックを備えた Strands Agents フレームワークを使用して、Memory 対応エージェントを構築します。このエージェントは、フェデレーション Identity 資格情報を使用して AgentCore Memory からメッセージを保存・取得することで会話コンテキストを維持します。

> **Memory が重要な理由**：AgentCore Runtime のセッションは一定時間後に期限切れになり、会話コンテキストが削除されます。会話を Memory に保存することで、セッション間で以前の情報が永続化され、長い休憩の後でもユーザーにシームレスな体験を提供できます。

### エージェントの機能

エージェントは以下を行います：
1. 各ユーザーメッセージとアシスタントメッセージを自動的に Memory に保存
2. 既存のセッションを継続する際に過去の会話履歴を取得
3. 同じユーザーとの複数のインタラクションにわたってコンテキストを維持
4. フェデレーション Identity の検証を通じて異なるユーザー間で会話を分離

### 実装の主要コンポーネント

#### 1. Memory フックプロバイダー
カスタムフックプロバイダーは以下を実装します：
- `on_agent_initialized`：エージェント開始時にトリガーされ、AgentCore Memory から会話履歴を取得
- `on_message_added`：新しいメッセージが会話に追加されたときにトリガーされ、AgentCore Memory に保存

#### 2. エージェント初期化
`initialize_agent` 関数：
- 正しいリージョンで Memory フックを設定
- 適切な状態変数（memory_id、actor_id、session_id）でエージェントを設定
- LLM のシステムプロンプトを設定

#### 3. フェデレーション資格情報
`get_aws_credentials_for_identity` 関数：
- Cognito ID トークンを一時的な AWS 資格情報と交換
- ユーザー固有の権限で AWS サービスにアクセスするために使用できる資格情報を返す

#### 4. エントリポイントハンドラー
runtime_memory_agent 関数：
- 入力ペイロードを解析し、ユーザーメッセージと ID トークンを抽出
- ID トークンを使用してフェデレーション資格情報を取得
- エージェントの初期化とセッション追跡を管理
- 適切なコンテキストでエージェントの呼び出しを処理
- Runtime 環境にフォーマットされた応答を返す

エージェントファイルを作成しましょう：

In [ ]:
%%writefile runtime_identity_memory_agent.py
import os
import jwt
import ast
import json
import boto3
import logging
import datetime

from strands import Agent
from jwt import PyJWKClient
from typing import Dict, Any
from strands.models import BedrockModel
from bedrock_agentcore.memory.session import MemorySessionManager
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory.constants import StrategyType, ConversationalMessage, MessageRole

# Configure detailed logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
)
logger = logging.getLogger("runtime-memory-agent")

# Initialize the agent core app
app = BedrockAgentCoreApp()

MODEL_ID = os.getenv('MODEL_ID')
MEMORY_ID = os.getenv('MEMORY_ID')
COGNITO_USER_POOL = os.getenv('COGNITO_USER_POOL')
IDENTITY_POOL_ID = os.getenv('IDENTITY_POOL_ID')
REGION = os.getenv('AWS_REGION')

# Global agent instance - will be initialized with first request
agent = None
memory_session_manager = None
actor_id = None

def get_aws_credentials_for_identity(identity_pool_id, id_token, region, user_pool_id):
    """
    Get temporary AWS credentials for a Cognito identity using a User Pool ID token
    """
    identity_client = boto3.client('cognito-identity', region_name=region)
    
    # Get ID from identity pool
    get_id_response = identity_client.get_id(
        IdentityPoolId=identity_pool_id,
        Logins={
            f'cognito-idp.{region}.amazonaws.com/{user_pool_id}': id_token
        }
    )
    identity_id = get_id_response['IdentityId']
    
    # Get credentials for the identity
    get_credentials_response = identity_client.get_credentials_for_identity(
        IdentityId=identity_id,
        Logins={
            f'cognito-idp.{region}.amazonaws.com/{user_pool_id}': id_token
        }
    )
    
    # Return the temporary credentials
    credentials = get_credentials_response['Credentials']
    return {
        'access_key_id': credentials['AccessKeyId'],
        'secret_key': credentials['SecretKey'],
        'session_token': credentials['SessionToken'],
        'expiration': credentials['Expiration'],
        'identity_id': identity_id
    }

class MemoryHookProvider(HookProvider):
    """Custom hook provider to integrate with Bedrock Memory"""
    
    def __init__(self):
        logger.info(f"Initializing MemoryHookProvider")
        # Use global memory_client that may have federated credentials
        self.memory_session_manager = memory_session_manager
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        logger.info("Agent initialization hook triggered")
        
        actor_id = event.agent.state.get("actor_id")
        session_id = event.agent.state.get("session_id")
        
        logger.info(f"State values - actor_id: {actor_id}, session_id: {session_id}")
        
        if not all([actor_id, session_id]):
            logger.warning("Missing required state values")
            return
        
        try:
            # Check if the session exists
            logger.info(f"Checking if session {session_id} exists...")
            session_exists = False
            try:
                events = self.memory_session_manager.list_events(
                    actor_id=actor_id,
                    session_id=session_id,
                    max_results=1
                )
                session_exists = len(events) > 0
                logger.info(f"Session exists: {session_exists}")
            except Exception as e:
                logger.warning(f"Error checking session existence: {e}")
                session_exists = False
            
            if not session_exists:
                logger.info(f"No existing conversation found for session {session_id}")
                return
            
            # Load conversation history
            logger.info(f"Loading conversation history for existing session {session_id}")
            recent_turns = self.memory_session_manager.get_last_k_turns(
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )            

            if recent_turns:
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns from memory")
                
                # Add messages to agent's conversation history
                for turn in reversed(recent_turns):
                    for message in turn:
                        role = message['role'].lower()  # 'user' or 'assistant'
                        parsed = ast.literal_eval(message['content']['text'])
                        content = parsed[0]['text']
                        
                        # Add to agent's message history
                        event.agent.messages.append({
                            "role": role,
                            "content": [{"text": content}]
                        })
                        logger.info(f"Added {role} message to history: {content[:50]}...")
                
                logger.info(f"✅ Added {len(event.agent.messages)} messages to conversation history")
            else:
                logger.info("No recent turns found for this session")
                    
        except Exception as e:
            logger.error(f"❌ Memory load error: {e}", exc_info=True)
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory"""
        logger.info("💬 Message added - storing in memory")
        
        actor_id = event.agent.state.get("actor_id")
        session_id = event.agent.state.get("session_id")
        
        if not all([actor_id, session_id]):
            logger.warning("Missing required state values")
            return
        
        try:
            messages = event.agent.messages
            last_message = messages[-1]
            message_content = str(last_message.get("content", ""))
            if last_message["role"] == "user":
                message_role = MessageRole.USER
            elif last_message["role"] == "assistant":
                message_role = MessageRole.ASSISTANT
            
            self.memory_session_manager.add_turns(
                actor_id=actor_id,
                session_id=session_id,
                messages=[ConversationalMessage(message_content, message_role)]
            )
            logger.info("✅ Message stored")
            
        except Exception as e:
            logger.error(f"❌ Error storing message: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        logger.info("Registering memory hooks")
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

def initialize_agent(actor_id, session_id):
    """Initialize the agent for first use"""
    global agent
    
    logger.info(f"Initializing agent for actor_id={actor_id}, session_id={session_id}")
    
    # Create model and memory hook
    logger.info(f"Creating model with ID: {MODEL_ID}")
    model = BedrockModel(model_id=MODEL_ID)
    logger.info(f"Creating memory hook with region: {REGION}")
    memory_hook = MemoryHookProvider()
    
    # Create agent with proper initial state
    logger.info("Creating agent with memory hook")
    agent = Agent(
        model=model,
        hooks=[memory_hook],
        system_prompt="You're a helpful agent. You can remember previous interactions within the same session. Be friendly and concise in your responses.",
        state={
            "actor_id": actor_id,
            "session_id": session_id
        }
    )
    logger.info(f"✅ Agent initialized with state: {agent.state.get()}")

def log_cognito_sub_from_token(id_token):
    """Log the sub value from the Cognito ID token"""
    try:
        # Decode without verification (just for logging)
        decoded = jwt.decode(
            id_token,
            options={"verify_signature": False, "verify_aud": False, "verify_exp": False}
        )
        logger.info(f"ID Token sub claim: {decoded.get('sub')}")
        return decoded.get('sub')
    except Exception as e:
        logger.error(f"Error decoding token: {e}")
        return None

@app.entrypoint
def runtime_memory_agent(payload, context):
    """
    Main entry point for the memory-enabled agent with identity federation
    
    Args:
        payload: The input payload containing user data
        context: The runtime context object containing session information
    """
    global agent, memory_session_manager
    
    # Log both payload and context info
    logger.info(f"Received payload: {payload}")
    logger.info(f"Context: {context}")
    logger.info(f"Context Auth: {context.request_headers.get('Authorization')}")
    
    # Extract and validate required values
    user_input = payload.get("prompt")
    id_token = payload.get("id_token")  # Get the ID token from payload
    auth_header = context.request_headers.get('Authorization')
    session_id = context.session_id
    
    # Validate required fields
    if user_input is None:
        error_msg = "❌ ERROR: Missing 'prompt' field in payload"
        logger.error(error_msg)
        return error_msg
    
    # Set up federated identity if ID token is provided
    if id_token and IDENTITY_POOL_ID:
        logger.info("ID token provided - setting up federated identity")
        
        # Get AWS credentials using the ID token
        user_credentials = get_aws_credentials_for_identity(
            identity_pool_id=IDENTITY_POOL_ID,
            id_token=id_token,
            region=REGION,
            user_pool_id=COGNITO_USER_POOL
        )
        
        # Set up actor_id
        logger.info(f"Identity Credentials: {user_credentials['identity_id']}")
        actor_id = user_credentials['identity_id']

        # Set up boto3 session with federated credentials
        session = boto3.Session(
            aws_access_key_id=user_credentials['access_key_id'],
            aws_secret_access_key=user_credentials['secret_key'],
            aws_session_token=user_credentials['session_token'],
            region_name=REGION
        )
        
        log_cognito_sub_from_token(id_token)

        # Create memory client with federated credentials 
        memory_session_manager = MemorySessionManager(
            memory_id = MEMORY_ID,
            region_name = REGION, 
            boto3_session = session
        )
        logger.info("✅ Successfully configured federated credentials for memory operations")
            
    
    # Initialize agent on first request
    if agent is None:
        logger.info("First request - initializing agent")
        initialize_agent(actor_id, session_id)
    else:
        logger.info("Using existing agent instance")
        # Update the session ID in case it changed
        if agent.state.get("session_id") != session_id:
            logger.info(f"Updating session ID to {session_id}")
            agent.state.set("session_id", session_id)
        if agent.state.get("actor_id") != actor_id:
            logger.info(f"Updating actor ID to {actor_id}")
            agent.state.set("actor_id", actor_id)
    
    # Invoke the agent with the user's input
    logger.info(f"Invoking agent with input: {user_input}")
    response = agent(user_input)
    response_text = response.message['content'][0]['text']
    logger.info(f"✅ Agent response: {response_text[:50]}…")
    
    return response_text

if __name__ == "__main__":
    logger.info("Starting AgentCore application")
    app.run()

## 4. AgentCore Runtime へのデプロイ

このセクションでは、エージェントを Amazon Bedrock AgentCore Runtime（スケーラビリティと運用の簡素化を提供するマネージドエージェントランタイム環境）にデプロイします。AgentCore Runtime はインフラストラクチャの複雑さを処理するため、デプロイメントの懸念ではなくエージェントのロジックに集中できます。

手動のサーバーセットアップと管理を必要とする従来のデプロイメント方法とは異なり、AgentCore Runtime はエージェントコンテナを AWS インフラストラクチャにデプロイし、呼び出し用の安全な HTTPS エンドポイントを提供します。このアプローチにより、エージェントは需要に応じてスケールし、本番環境で確実に動作できます。

> 💡 **ヒント**：AgentCore スターターツールキットは、IAM ロール、ECR リポジトリ、コンテナビルドなど、すべての複雑なデプロイメント手順を処理します。

### デプロイメントの設定

デプロイメント設定をセットアップしましょう：

In [ ]:
iam_role = create_agentcore_role(agent_name=f"runtime_memory_agent_{unique_id}", region=REGION)

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import time

agentcore_runtime = Runtime()
agent_name = f"runtime_memory_agent_{unique_id}"

response = agentcore_runtime.configure(
    entrypoint="runtime_identity_memory_agent.py", 
    execution_role = iam_role["Role"]["RoleName"],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name=agent_name,
    non_interactive=True, 
    memory_mode="NO_MEMORY",
    idle_timeout = 60,
    request_header_configuration = {"requestHeaderAllowlist": ["Authorization"]},
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": cognito_config.get("discovery_url"),
            "allowedClients": [cognito_config.get("client_id")]
        }
    }
)
response

### エージェントの起動

AgentCore Runtime にエージェントを起動しましょう。このステップでは、設定したエージェントを取得し、AgentCore Runtime のマネージドインフラストラクチャにデプロイします。このプロセス中に、エージェントに必要な基本的な環境変数（先ほど作成した Memory ID、使用するモデル ID、AWS リージョン、認証用の Cognito User Pool ID）も渡しています。

デプロイ後、エージェントはユーザーメッセージで呼び出せるセキュアなエンドポイントを通じてアクセス可能になります。エンドポイントは AgentCore Identity インバウンド認証で保護され、許可されたユーザーのみがエージェントにアクセスできることを保証します。

In [ ]:
launch_result = agentcore_runtime.launch(
    env_vars={
        "MEMORY_ID": MEMORY_ID,
        "MODEL_ID": "us.anthropic.claude-haiku-4-5-20251001-v1:0",
        "AWS_REGION": REGION,
        "COGNITO_USER_POOL": cognito_config["pool_id"],
        "IDENTITY_POOL_ID":cognito_config["identity_pool_id"]
    }
)

### デプロイメントステータスの確認

エージェントのデプロイメントステータスを確認しましょう。AgentCore Runtime がコンテナをビルドし、必要なリソースをプロビジョニングし、エージェントを AWS インフラストラクチャにデプロイするため、これには数分かかることがあります。デプロイメントが完了するまで 10 秒ごとにステータスをポーリングします。

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Current status: {status}")

if status == 'READY':
    print("✅ Agent successfully deployed!")
else:
    print(f"❌ Deployment ended with status: {status}")

account_id = status_response.config.account
role_arn = status_response.config.execution_role
role_name = role_arn.split("/")[-1]
print(f"Account ID is {account_id}")
print(f"Agent role is {role_arn}")
print(f"Role name is {role_name}")

## 5. エージェントのテスト

エージェントがデプロイされたので、メッセージを送信して以前のインタラクションを記憶できることを確認してテストしましょう。また、異なるユーザーが分離された Memory コンテキストを持つことをテストし、あるユーザーの会話が別のユーザーに見えないことを確認します。

**セッション管理に関する重要な注意事項**

- **セッション管理**：AgentCore Runtime はセッション ID が提供されない場合に自動的に生成しますが、アプリケーションでセッション ID を明示的に管理することをお勧めします。これにより、以下をより適切に制御できます：
  - セッションタイムアウト後の会話の継続
  - 適切な場合（例：ユーザーが新しい会話を開始）に新しいセッションを作成
  - 同じユーザーとの複数の並行会話の処理
  - アプリケーションのニーズに基づいたセッション有効期限ポリシーの実装

- **Memory の永続性**：AgentCore Runtime でセッションが期限切れになっても、エージェントは同じユーザーで新しいセッションが開始されたときに AgentCore Memory から以前の会話を取得できます。

In [ ]:
import time

def test_user_memory_isolation_with_federated_identity():
    """
    Test user memory isolation using federated identity credentials.
    """
    print("\n" + "=" * 50)
    print("USER MEMORY ISOLATION WITH FEDERATED IDENTITY TEST")
    print("=" * 50)
    
    # Extract bearer tokens and ID tokens
    testuser1_token = cognito_config["bearer_tokens"]["testuser1"]
    testuser2_token = cognito_config["bearer_tokens"]["testuser2"]
    testuser1_id_token = cognito_config["id_tokens"]["testuser1"]
    testuser2_id_token = cognito_config["id_tokens"]["testuser2"]
    
    # Create unique session IDs for each test phase
    user1_session_id = f"memory-agent-session-user1-{int(time.time())}"
    user2_session_id = f"memory-agent-session-user2-{int(time.time())}"
    
    # PHASE 1: Test with user1's memory persistence
    print("\n" + "=" * 50)
    print("PHASE 1: USER 1 MEMORY PERSISTENCE TEST")
    print("=" * 50)

    # Step 1: User 1 shares initial information
    print("\n" + "-" * 50)
    print("STEP 1: User 1 shares information")
    print("-" * 50)

    user1_prompt1 = "My name is Dani and my favorite color is blue."
    response1 = agentcore_runtime.invoke(
        {
            "prompt": user1_prompt1,
            "id_token": testuser1_id_token
        },
        session_id=user1_session_id,
        bearer_token=testuser1_token
    )
    print(f"User 1 prompt: \"{user1_prompt1}\"")
    print(f"User 1 response: \"{response1['response']}\"")

    # Wait for session to terminate (75 seconds)
    print("\nWaiting 75 seconds for session to terminate...")
    time.sleep(75)

    # Step 2: User 1 asks to recall information
    print("\n" + "-" * 50)
    print("STEP 2: User 1 recalls information (should succeed)")
    print("-" * 50)

    user1_prompt2 = "What is my name and favorite color?"
    response2 = agentcore_runtime.invoke(
        {
            "prompt": user1_prompt2,
            "id_token": testuser1_id_token
        },
        session_id=user1_session_id,
        bearer_token=testuser1_token
    )
    print(f"User 1 prompt: \"{user1_prompt2}\"")
    print(f"User 1 response: \"{response2['response']}\"")

    # PHASE 2: Test user2 memory isolation
    print("\n" + "=" * 50)
    print("PHASE 2: USER 2 MEMORY ISOLATION TEST")
    print("=" * 50)

    # Step 3: User 2 shares information
    print("\n" + "-" * 50)
    print("STEP 3: User 2 shares information")
    print("-" * 50)

    user2_prompt1 = "My name is Paula and my favorite color is pink."
    response3 = agentcore_runtime.invoke(
        {
            "prompt": user2_prompt1,
            "id_token": testuser2_id_token
        },
        session_id=user2_session_id,
        bearer_token=testuser2_token
    )
    print(f"User 2 prompt: \"{user2_prompt1}\"")
    print(f"User 2 response: \"{response3['response']}\"")

    # Wait for session to terminate
    print("\nWaiting 75 seconds for session to terminate...")
    time.sleep(75)

    # Step 4: User 2 tries to recall (should only see their own info)
    print("\n" + "-" * 50)
    print("STEP 4: User 2 recalls information (should see only their data)")
    print("-" * 50)

    user2_prompt2 = "What is my name and favorite color?"
    response4 = agentcore_runtime.invoke(
        {
            "prompt": user2_prompt2,
            "id_token": testuser2_id_token
        },
        session_id=user2_session_id,
        bearer_token=testuser2_token
    )
    print(f"User 2 prompt: \"{user2_prompt2}\"")
    print(f"User 2 response: \"{response4['response']}\"")
    print("\n✅ Each user should only see their own information, demonstrating memory isolation")

In [ ]:
test_user_memory_isolation_with_federated_identity()

## 主要な概念

このチュートリアルでは、AgentCore で Memory 対応エージェントを構築するためのいくつかの重要な概念を学習しました：

1. **Memory 統合**：AgentCore Memory を使用してセッション間で会話履歴を保存し、セッションが期限切れになってもエージェントが時間の経過とともにコンテキストを維持できるようにする方法。

2. **セッション管理**：セッション ID を使用して会話を整理し、ユーザーが戻ってきたときに関連する履歴を取得し、シームレスな体験を作成する方法。

3. **AgentCore Runtime デプロイメント**：スケーリング、セキュリティ、インフラストラクチャ管理を自動的に処理する本番ランタイム環境にエージェントをデプロイする方法。

4. **Memory フック**：エージェントのライフサイクルの特定のポイントで会話履歴を保存・取得できる Memory サービスと統合するカスタムフックを実装する方法。

5. **フェデレーション Identity とプライバシー**：Cognito Identity Pool とフェデレーション資格情報を使用して、AWS IAM を通じて各ユーザーの会話履歴がプライベートであり、他のユーザーから分離されていることを確保する方法。

これらの概念は、永続的な Memory と洗練された会話管理機能を持つより複雑なエージェントを構築するための基盤を提供します。

## クリーンアップ（オプション）

このチュートリアルで作成したリソースが不要になった場合は、不要な AWS 料金を避けるためにクリーンアップできます。

In [ ]:
# Only run this cell if you want to delete all resources

# 1. Delete the AgentCore Runtime
if 'launch_result' in locals() and hasattr(launch_result, 'agent_id'):
    try:
        agentcore_control_client = boto3.client(
            'bedrock-agentcore-control',
            region_name=REGION
        )
        
        runtime_delete_response = agentcore_control_client.delete_agent_runtime(
            agentRuntimeId=launch_result.agent_id,
        )
        print(f"✅ Deleted AgentCore Runtime: {launch_result.agent_id}")
    except Exception as e:
        print(f"❌ Error deleting AgentCore Runtime: {e}")
else:
    print("No AgentCore Runtime to delete")

# 2. Delete the ECR repository
if 'launch_result' in locals() and hasattr(launch_result, 'ecr_uri'):
    try:
        ecr_client = boto3.client(
            'ecr',
            region_name=REGION
        )
        
        repository_name = launch_result.ecr_uri.split('/')[1]
        response = ecr_client.delete_repository(
            repositoryName=repository_name,
            force=True  # Force deletion even if it contains images
        )
        print(f"✅ Deleted ECR repository: {repository_name}")
    except Exception as e:
        print(f"❌ Error deleting ECR repository: {e}")
else:
    print("No ECR repository to delete")

# 3. Delete the memory resource
try:
    memory_manager.delete_memory(memory_id=MEMORY_ID)
    print(f"✅ Deleted memory resource: {MEMORY_ID}")
except Exception as e:
    print(f"❌ Error deleting memory resource: {e}")

# 4. Delete the Cognito User Pool and associated resources
if 'cognito_config' in locals() and cognito_config and 'pool_id' in cognito_config:
    try:
        cognito_client = boto3.client('cognito-idp', region_name=REGION)
        
        # Get the user pool ID
        pool_id = cognito_config['pool_id']
        
        # List and delete all user pool clients
        clients_response = cognito_client.list_user_pool_clients(
            UserPoolId=pool_id,
            MaxResults=60
        )
        
        for client in clients_response.get('UserPoolClients', []):
            client_id = client['ClientId']
            cognito_client.delete_user_pool_client(
                UserPoolId=pool_id,
                ClientId=client_id
            )
            print(f"✅ Deleted User Pool Client: {client_id}")
        
        # Delete the user pool itself
        cognito_client.delete_user_pool(
            UserPoolId=pool_id
        )
        print(f"✅ Deleted Cognito User Pool: {pool_id}")
        
    except Exception as e:
        print(f"❌ Error deleting Cognito resources: {e}")
else:
    print("No Cognito resources to delete")

# 5. Function to delete IAM role and all its versions
def delete_iam_role(role_identifier, region=REGION):
    """
    Deletes an IAM role including all attached policies and versions
    
    Args:
        role_identifier (str): The ARN or name of the IAM role
        region (str): AWS region
    """
    try:
        iam_client = boto3.client('iam', region_name=region)
        
        # Determine if the identifier is an ARN or a role name
        if role_identifier.startswith('arn:aws:iam::'):
            # Extract role name from ARN
            role_name = role_identifier.split('/')[-1]
        else:
            role_name = role_identifier
            
        print(f"Attempting to delete IAM role: {role_name}")
            
        # 1. Detach all managed policies
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)
        for policy in attached_policies.get('AttachedPolicies', []):
            iam_client.detach_role_policy(
                RoleName=role_name,
                PolicyArn=policy['PolicyArn']
            )
            print(f"✅ Detached managed policy: {policy['PolicyArn']}")
            
        # 2. Delete all inline policies
        inline_policies = iam_client.list_role_policies(RoleName=role_name)
        for policy_name in inline_policies.get('PolicyNames', []):
            iam_client.delete_role_policy(
                RoleName=role_name,
                PolicyName=policy_name
            )
            print(f"✅ Deleted inline policy: {policy_name}")
            
        # 3. Delete any instance profiles associated with the role
        instance_profiles = iam_client.list_instance_profiles_for_role(RoleName=role_name)
        for profile in instance_profiles.get('InstanceProfiles', []):
            iam_client.remove_role_from_instance_profile(
                InstanceProfileName=profile['InstanceProfileName'],
                RoleName=role_name
            )
            print(f"✅ Removed role from instance profile: {profile['InstanceProfileName']}")
            
        # 4. Finally delete the role
        iam_client.delete_role(RoleName=role_name)
        print(f"✅ Successfully deleted IAM role: {role_name}")
        
    except Exception as e:
        print(f"❌ Error deleting IAM role: {e}")


delete_iam_role(role_arn)

print("\n✅ Cleanup complete")